# Monte Carlo simulation Coefficient 3

In [2]:
import os
import sys
import numpy as np
import scipy.sparse as sparse
import scipy.stats as stats
import random
import csv

%matplotlib notebook
import matplotlib.pyplot as plt
from visualize import drawCoefficient
from data import * 

from gridlod import interp, coef, util, fem, world, linalg, femsolver
import pg_rand, femsolverCoarse, buildcoef2d
from gridlod.world import World


## Result function

In [7]:
def result(pglod, world, CoefClass, A, f, MC=1, numberofcorrectors=20, MCplot=None, ROOT=None):
    NWorldFine = world.NWorldFine
    NWorldCoarse = world.NWorldCoarse
    NCoarseElement = world.NCoarseElement
    
    boundaryConditions = world.boundaryConditions
    NpFine = np.prod(NWorldFine+1)
    NpCoarse = np.prod(NWorldCoarse+1)
    
    #### initial #####
    xmLoda = []
    xmVcLoda = []
    xmLodVcLoda = []
    
    ems = []
    
    plottingx = []
    plottingy = []
    plottingz = []
    
    plotting2x = []
    plotting2y = []
    plotting2z = []
    
    plotting3x = []
    plotting3y = []
    plotting3z = []
    
    for i in range(0,MC):
        print '_____Sample__ ' + str(i+1) + '/' + str(MC) + ' ____' 
        xmLod, xmVcLod, xmLodVcLod = VcLod(pglod, world, CoefClass, f, prob=100, numberofcorrectors=numberofcorrectors, m=i)
        xmLoda.append(xmLod)
        xmVcLoda.append(xmVcLod)
        xmLodVcLoda.append(xmLodVcLod)
    
        muLod = 0
        muVcLod = 0
        muLodVcLod = 0
        if i == 0:
            continue
            
        for j in range(0,i+1):
            muLod += xmLoda[j]
            muVcLod += xmVcLoda[j]
            muLodVcLod += xmLodVcLoda[j]
    
        muLod /= i+1
        muVcLod /= i+1
        muLodVcLod /= i+1
    
        sig2Lod = 0
        sig2VcLod = 0
        sig2LodVcLod = 0
    
        for j in range(0,i+1):
            sig2Lod += (xmLoda[j]-muLod)**(2)
            sig2VcLod += (xmVcLoda[j]-muVcLod)**(2)
            sig2LodVcLod += (xmLodVcLoda[j]-muLodVcLod)**(2)
        
        sig2Lod /= i
        sig2VcLod /= i
        sig2LodVcLod /= i
    
        a = [np.sqrt(sig2Lod)/np.sqrt(i+1)*1.96,np.sqrt(sig2VcLod)/np.sqrt(i+1)*1.96,np.sqrt(sig2LodVcLod)/np.sqrt(i+1)*1.96]
        mum = [muLod,muVcLod,muLodVcLod]
        
        plottingx.append(mum[0]-a[0])
        plottingy.append(mum[0])
        plottingz.append(mum[0]+a[0])
        
        plotting2x.append(mum[1]-a[1])
        plotting2y.append(mum[1])
        plotting2z.append(mum[1]+a[1])

        plotting3x.append(mum[1]-a[1])
        plotting3y.append(mum[1])
        plotting3z.append(mum[1]+a[1])
        
        ems.append(i+1)
        Matrix = CoefClass.Matrix.flatten()
        safer(ROOT, mum, a, plottingx, plottingy, plottingz, plotting2x, plotting2y, plotting2z, plotting3x, plotting3y, plotting3z, ems, Matrix)
        
    return a,mum

## Error function

In [4]:
def Errors(world, R, uLod, uVcLod):
    NWorldFine = world.NWorldFine
    NpFine = np.prod(NWorldFine+1)
    
    ANew = R.flatten()
    boundaryConditions = world.boundaryConditions
    f_fine = np.ones(NpFine)
    uFineFem, AFine, MFine = femsolver.solveFine(world, ANew, f_fine, None, boundaryConditions)
    
    errorLod = np.sqrt(np.dot(uFineFem - uLod, MFine*(uFineFem - uLod))) / np.sqrt(np.dot(uFineFem, MFine*uFineFem))
    errorVcLod = np.sqrt(np.dot(uFineFem - uVcLod, MFine*(uFineFem - uVcLod))) / np.sqrt(np.dot(uFineFem, MFine*uFineFem))
    errorLodVcLod = np.sqrt(np.dot(uVcLod - uLod, MFine*(uVcLod - uLod))) / np.sqrt(np.dot(uLod, MFine*uLod))
    
    return errorLod, errorVcLod, errorLodVcLod

## VC-LOD

In [5]:
def VcLod(pglod, world, CoefClass,f, prob=100, numberofcorrectors=20, m=0):
    NWorldFine = world.NWorldFine
    NWorldCoarse = world.NWorldCoarse
    NCoarseElement = world.NCoarseElement

    boundaryConditions = world.boundaryConditions
    NpFine = np.prod(NWorldFine+1)
    NpCoarse = np.prod(NWorldCoarse+1)

    R = CoefClass.RandomVanish( probfactor      = prob,
                                PartlyVanish    = None,
                                Original        = True)

    #new Coefficient
    ANew = R.flatten()
    Anew = coef.coefficientFine(NWorldCoarse, NCoarseElement, ANew)

    ###### tolerance = 0 ######
    vis, eps = pglod.updateCorrectors(Anew, 0, f, 1, clearFineQuantities=False, mc=True)
    print 'Affected correctors: ' + str(np.sum(vis))
    KFull = pglod.assembleMsStiffnessMatrix()
    MFull = fem.assemblePatchMatrix(NWorldCoarse, world.MLocCoarse)
    free  = util.interiorpIndexMap(NWorldCoarse)

    bFull = MFull*f
    KFree = KFull[free][:,free]
    bFree = bFull[free]

    xFree = sparse.linalg.spsolve(KFree, bFree)
    basis = fem.assembleProlongationMatrix(NWorldCoarse, NCoarseElement)

    basisCorrectors = pglod.assembleBasisCorrectors()
    modifiedBasis = basis - basisCorrectors

    xFull = np.zeros(NpCoarse)
    xFull[free] = xFree
    uCoarse = xFull
    uLod = modifiedBasis*xFull

    ##### tolerance = certain ######
    eps = filter(lambda x: x!=0, eps)
    eps.sort()
    epssize = np.size(eps)
    until = int(round(numberofcorrectors/100. * epssize,0))
    if epssize != 0:
        until = int(round(until * 256./epssize,0))
    tolrev = []
    for i in range(epssize-1,-1,-1):
        tolrev.append(eps[i])
    
    if epssize == 0:
        print 'nothing to update'
        xmLod, xmVcLod, xmLodVcLod = Errors(world, R, uLod, uLod)
    
    else:
        if until >= epssize:
            until = epssize-1
        tol = tolrev[until]
    
        vistol = pglod.updateCorrectors(Anew, tol, f, clearFineQuantities=False)
        print 'Updated correctors: ' + str(np.sum(vistol))
        KFull = pglod.assembleMsStiffnessMatrix()
        MFull = fem.assemblePatchMatrix(NWorldCoarse, world.MLocCoarse)
        free  = util.interiorpIndexMap(NWorldCoarse)

        bFull = MFull*f
        KFree = KFull[free][:,free]
        bFree = bFull[free]

        xFree = sparse.linalg.spsolve(KFree, bFree)
        basis = fem.assembleProlongationMatrix(NWorldCoarse, NCoarseElement)

        basisCorrectors = pglod.assembleBasisCorrectors()
        modifiedBasis = basis - basisCorrectors

        xFull = np.zeros(NpCoarse)
        xFull[free] = xFree
        uCoarse = xFull
        uVcLod = modifiedBasis*xFull

        xmLod, xmVcLod, xmLodVcLod = Errors(world, R, uLod, uVcLod)

    return xmLod, xmVcLod, xmLodVcLod

In [6]:
#background
bg = 0.05
#values
val = 1
random.seed(20)
#fine World
NWorldFine = np.array([256, 256])
NpFine = np.prod(NWorldFine+1)                                                                               

#coarse World
NWorldCoarse = np.array([16,16])
NpCoarse = np.prod(NWorldCoarse+1)

#ratio between Fine and Coarse
NCoarseElement = NWorldFine/NWorldCoarse

boundaryConditions = np.array([[0, 0],
                               [0, 0]])

world = World(NWorldCoarse, NCoarseElement, boundaryConditions)

#righthandside
f = np.ones(NpCoarse)

#coefficient 3
CoefClass = buildcoef2d.Coefficient2d(NWorldFine, 
                        bg                  = bg, 
                        val                 = val, 
                        length              = 8, 
                        thick               = 2, 
                        space               = 4, 
                        probfactor          = 1, 
                        right               = 0, 
                        down                = 0, 
                        diagr1              = 0, 
                        diagr2              = 0, 
                        diagl1              = 0, 
                        diagl2              = 1, 
                        LenSwitch           = None, 
                        thickSwitch         = None, 
                        equidistant         = None, 
                        ChannelHorizontal   = None, 
                        ChannelVertical     = None,
                        BoundarySpace       = True)

A = CoefClass.BuildCoefficient()
ABase = A.flatten()
        
plt.figure("OriginalCoefficient")
drawCoefficient(NWorldFine, ABase)
plt.title('Original coefficient')

k = 4

###### precompute #######
NWorldFine = world.NWorldFine
NWorldCoarse = world.NWorldCoarse
NCoarseElement = world.NCoarseElement

boundaryConditions = world.boundaryConditions
NpFine = np.prod(NWorldFine+1)
NpCoarse = np.prod(NWorldCoarse+1)

#interpolant
IPatchGenerator = lambda i, N: interp.L2ProjectionPatchMatrix(i, N, NWorldCoarse, NCoarseElement, boundaryConditions)

#old Coefficient (need flatten form)
ABase = A.flatten()
Aold = coef.coefficientFine(NWorldCoarse, NCoarseElement, ABase)

pglod = pg_rand.VcPetrovGalerkinLOD(Aold, world, k, IPatchGenerator, 0)
pglod.originCorrectors(clearFineQuantities=False)

#correctors that we will update in %
numberofcorrectors = 10

MC = 400
ROOT = '../test_data/MonteCarlo/Coef3/p20'
a, mum = result(pglod, world, CoefClass, A, f, MC, numberofcorrectors, MCplot=True, ROOT=ROOT)

print 'p = '+ str(10) + '%'
print 'uLod  ' + str(mum[0]) + ' +- ' + str(a[0])
print 'uVcLod ' + str(mum[1]) + ' +- ' + str(a[1])
print 'uLodVcLod ' + str(mum[2]) + ' +- ' + str(a[2])


<IPython.core.display.Javascript object>

_____Sample__ 1/400 ____
Affected correctors: 227.0
Updated correctors: 26.0
_____Sample__ 2/400 ____
Affected correctors: 187.0
Updated correctors: 26.0
_____Sample__ 3/400 ____
Affected correctors: 213.0
Updated correctors: 25.0
_____Sample__ 4/400 ____
Affected correctors: 251.0
Updated correctors: 25.0
_____Sample__ 5/400 ____
Affected correctors: 180.0
Updated correctors: 26.0
_____Sample__ 6/400 ____
Affected correctors: 199.0
Updated correctors: 26.0
_____Sample__ 7/400 ____
Affected correctors: 158.0
Updated correctors: 26.0
_____Sample__ 8/400 ____
Affected correctors: 161.0
Updated correctors: 25.0
_____Sample__ 9/400 ____
Affected correctors: 171.0
Updated correctors: 25.0
_____Sample__ 10/400 ____
Affected correctors: 218.0
Updated correctors: 26.0
_____Sample__ 11/400 ____
Affected correctors: 240.0
Updated correctors: 26.0
_____Sample__ 12/400 ____
Affected correctors: 152.0
Updated correctors: 25.0
_____Sample__ 13/400 ____
Affected correctors: 157.0
Updated correctors: 

Affected correctors: 172.0
Updated correctors: 25.0
_____Sample__ 107/400 ____
Affected correctors: 247.0
Updated correctors: 26.0
_____Sample__ 108/400 ____
Affected correctors: 156.0
Updated correctors: 26.0
_____Sample__ 109/400 ____
Affected correctors: 187.0
Updated correctors: 26.0
_____Sample__ 110/400 ____
Affected correctors: 206.0
Updated correctors: 26.0
_____Sample__ 111/400 ____
Affected correctors: 141.0
Updated correctors: 25.0
_____Sample__ 112/400 ____
Affected correctors: 184.0
Updated correctors: 25.0
_____Sample__ 113/400 ____
Affected correctors: 208.0
Updated correctors: 26.0
_____Sample__ 114/400 ____
Affected correctors: 254.0
Updated correctors: 25.0
_____Sample__ 115/400 ____
Affected correctors: 188.0
Updated correctors: 26.0
_____Sample__ 116/400 ____
Affected correctors: 247.0
Updated correctors: 26.0
_____Sample__ 117/400 ____
Affected correctors: 202.0
Updated correctors: 25.0
_____Sample__ 118/400 ____
Affected correctors: 222.0
Updated correctors: 25.0


Affected correctors: 181.0
Updated correctors: 25.0
_____Sample__ 211/400 ____
Affected correctors: 241.0
Updated correctors: 25.0
_____Sample__ 212/400 ____
Affected correctors: 199.0
Updated correctors: 26.0
_____Sample__ 213/400 ____
Affected correctors: 201.0
Updated correctors: 25.0
_____Sample__ 214/400 ____
Affected correctors: 219.0
Updated correctors: 26.0
_____Sample__ 215/400 ____
Affected correctors: 149.0
Updated correctors: 26.0
_____Sample__ 216/400 ____
Affected correctors: 164.0
Updated correctors: 25.0
_____Sample__ 217/400 ____
Affected correctors: 227.0
Updated correctors: 26.0
_____Sample__ 218/400 ____
Affected correctors: 134.0
Updated correctors: 25.0
_____Sample__ 219/400 ____
Affected correctors: 225.0
Updated correctors: 26.0
_____Sample__ 220/400 ____
Affected correctors: 182.0
Updated correctors: 25.0
_____Sample__ 221/400 ____
Affected correctors: 206.0
Updated correctors: 26.0
_____Sample__ 222/400 ____
Affected correctors: 193.0
Updated correctors: 25.0


Affected correctors: 224.0
Updated correctors: 25.0
_____Sample__ 315/400 ____
Affected correctors: 184.0
Updated correctors: 25.0
_____Sample__ 316/400 ____
Affected correctors: 221.0
Updated correctors: 25.0
_____Sample__ 317/400 ____
Affected correctors: 213.0
Updated correctors: 25.0
_____Sample__ 318/400 ____
Affected correctors: 224.0
Updated correctors: 25.0
_____Sample__ 319/400 ____
Affected correctors: 188.0
Updated correctors: 26.0
_____Sample__ 320/400 ____
Affected correctors: 220.0
Updated correctors: 26.0
_____Sample__ 321/400 ____
Affected correctors: 219.0
Updated correctors: 26.0
_____Sample__ 322/400 ____
Affected correctors: 81.0
Updated correctors: 25.0
_____Sample__ 323/400 ____
Affected correctors: 172.0
Updated correctors: 25.0
_____Sample__ 324/400 ____
Affected correctors: 170.0
Updated correctors: 26.0
_____Sample__ 325/400 ____
Affected correctors: 166.0
Updated correctors: 26.0
_____Sample__ 326/400 ____
Affected correctors: 165.0
Updated correctors: 26.0
_

_____Sample__ 17/400 ____
Affected correctors: 220.0
Updated correctors: 51.0
_____Sample__ 18/400 ____
Affected correctors: 167.0
Updated correctors: 51.0
_____Sample__ 19/400 ____
Affected correctors: 161.0
Updated correctors: 51.0
_____Sample__ 20/400 ____
Affected correctors: 243.0
Updated correctors: 52.0
_____Sample__ 21/400 ____
Affected correctors: 191.0
Updated correctors: 51.0
_____Sample__ 22/400 ____
Affected correctors: 151.0
Updated correctors: 51.0
_____Sample__ 23/400 ____
Affected correctors: 193.0
Updated correctors: 52.0
_____Sample__ 24/400 ____
Affected correctors: 158.0
Updated correctors: 52.0
_____Sample__ 25/400 ____
Affected correctors: 201.0
Updated correctors: 51.0
_____Sample__ 26/400 ____
Affected correctors: 151.0
Updated correctors: 51.0
_____Sample__ 27/400 ____
Affected correctors: 222.0
Updated correctors: 51.0
_____Sample__ 28/400 ____
Affected correctors: 241.0
Updated correctors: 51.0
_____Sample__ 29/400 ____
Affected correctors: 132.0
Updated cor

IndexError: list index out of range

In [ ]:
#correctors that we will update in %
numberofcorrectors = 20

ROOT = '../test_data/MonteCarlo/Coef3/p40'
a, mum = result(pglod, world, CoefClass, A, f, MC, numberofcorrectors, MCplot=True, ROOT=ROOT)

print 'p = '+ str(20) + '%'
print 'uLod  ' + str(mum[0]) + ' +- ' + str(a[0])
print 'uVcLod ' + str(mum[1]) + ' +- ' + str(a[1])
print 'uLodVcLod ' + str(mum[2]) + ' +- ' + str(a[2])


_____Sample__ 1/400 ____
Affected correctors: 55.0
Updated correctors: 51.0
_____Sample__ 2/400 ____
Affected correctors: 245.0
Updated correctors: 51.0
_____Sample__ 3/400 ____
Affected correctors: 230.0
Updated correctors: 51.0
_____Sample__ 4/400 ____
Affected correctors: 181.0
Updated correctors: 51.0
_____Sample__ 5/400 ____
Affected correctors: 149.0
Updated correctors: 52.0
_____Sample__ 6/400 ____
